In [94]:
from google.colab import drive

drive.mount('/content/drive')
!ls '/content/drive/My Drive/DSL Final Project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Blog Post'			     'List of users and number of posts.gdoc'
'Blog Post.gdoc'		     'List of users and number of posts.txt'
 body_parts.txt			      Planning.gdoc
 Classifier.ipynb		     'Predicting Popular Reddit Posts.ipynb'
'Copy of Models.ipynb'		      Scraper.ipynb
 data_for_NN.pickle		      supernaturalwords.txt
 data.pickle			      trainer.ipynb
'Data Science Presentation.gslides'   violentwords.txt
 featured_data.pickle


In [0]:
from sklearn.linear_model import Lasso
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [0]:
file = open('/content/drive/My Drive/DSL Final Project/featured_data.pickle', 'rb')
df = pickle.load(file)
df = df.reset_index()
df = df.drop(['index'],axis=1)

In [0]:
dropped = []
for i in range(len(df)):
  if df.iloc[i,3]==None or 'Removed' in df.iloc[i,3]:
    dropped.append(i)
df = df.drop(dropped)
df = df.reset_index()
df = df.drop(['index'],axis=1)

In [0]:
flairs = pd.get_dummies(df['is_series'])
df["time_of_day"] = df["time_of_day"].apply(lambda x: "time_of_day: " + str(x))
time_of_day = pd.get_dummies(df['time_of_day'])
df["day_of_week"] = df["day_of_week"].apply(lambda x: "day_of_week: " + str(x))
day_of_week = pd.get_dummies(df['day_of_week'])
df["year_posted"] = df["year_posted"].apply(lambda x: "20" + str(x))
year_posted = pd.get_dummies(df['year_posted'])
df["month_posted"] = df["month_posted"].apply(lambda x: "month_posted: " + str(x))
month_posted = pd.get_dummies(df['month_posted'])
df = df.drop(columns=['author','created_utc','post_content','is_series','title','time_of_day','month_posted','year_posted','day_of_week'])
df = pd.concat([df,flairs,time_of_day,day_of_week,year_posted,month_posted],axis=1)

In [98]:
df_pop = df[df['num_upvotes']>=1000]
df_unpop = df[df['num_upvotes']<1000]
df_pop['is_popular'] = 1
df_unpop['is_popular'] = 0
df = pd.concat([df_pop,df_unpop])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [93]:
print(df_pop)

        num_upvotes  story_wordcount  ...  month_posted: 12  is_popular
0              1978              446  ...                 1           1
1              1494              755  ...                 1           1
76             1186              514  ...                 1           1
320            2385             5148  ...                 1           1
321            1043             4113  ...                 1           1
...             ...              ...  ...               ...         ...
179861         1390             1063  ...                 0           1
179976         5299             5116  ...                 0           1
179977         2513             5135  ...                 0           1
179978         1870             2234  ...                 0           1
180095         1855             1160  ...                 0           1

[3789 rows x 80 columns]


In [0]:
data = df.values
y = data[:,-1]
y = y[:,np.newaxis]
X = data[:,1:-1]
scalerX = StandardScaler()
scalery = StandardScaler()
scaled_X = scalerX.fit_transform(X)
#scaled_y = scalery.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.5, random_state=42)

In [81]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=0.8).fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [84]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,clf.predict(X_test))

array([[1465,  424],
       [ 506, 1394]])

In [82]:
from sklearn.metrics import roc_auc_score
y_pred = clf.predict_proba(X_test)
roc_auc_score(y_test,y_pred[:,0])

0.1726070602658048

In [73]:
print(clf.score(X_test,y_test))

0.7545526524148852


In [77]:
from xgboost import XGBClassifier
xgb = XGBClassifier().fit(X_train,y_train)
y_pred = xgb.predict_proba(X_test)
roc_auc_score(y_test,y_pred[:,0])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.13030174695606142

In [79]:
print(y_pred)

[[0.19933516 0.80066484]
 [0.8989383  0.10106169]
 [0.18775463 0.81224537]
 ...
 [0.76652586 0.23347414]
 [0.38726556 0.61273444]
 [0.9631055  0.03689449]]


In [0]:
df_pop = df[df['num_upvotes']>=19]
df_unpop = df[df['num_upvotes']<19]
df_pop['is_popular'] = 1
df_unpop['is_popular'] = 0
df = pd.concat([df_pop,df_unpop])

In [69]:
from sklearn.utils import resample
df_pop = df[df['num_upvotes']>=1000]
df_unpop = df[df['num_upvotes']<1000]
df_unpop = resample(df_unpop,
                          replace=True, 
                          n_samples=len(df_pop), 
                          random_state=27)
df_pop['is_popular'] = 1
df_unpop['is_popular'] = 0
df = pd.concat([df_pop,df_unpop])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
print(df)

        num_upvotes  story_wordcount  ...  month_posted: 12  is_popular
0              1978              446  ...                 1           1
1              1494              755  ...                 1           1
76             1186              514  ...                 1           1
320            2385             5148  ...                 1           1
321            1043             4113  ...                 1           1
...             ...              ...  ...               ...         ...
43906             5              519  ...                 0           0
81341             1                1  ...                 0           0
151082            0             2406  ...                 0           0
23810            17                1  ...                 1           0
3553             26              866  ...                 0           0

[7578 rows x 80 columns]
